# Automation task 1

In [ ]:
# python 
# selenium (headless browser automation)
# Azure Functions (to schedule / trigger the task)
# Azure Blob Storage (to store downloaded PDFs)
# Azure Key Vault (security)

1. Read IDRSSD list 
2. Use Selenium to Navigate URL 
3. Enter IDRSSD and select report date 
4. Generate -> Wait for PDF to render 
5. Download PDF 
6. Save the file to Azure Blob Storage
7. Log/report success/failure 

In [ ]:
# version 2 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests, os, time
from selenium.webdriver.support.ui import Select
import dotenv
dotenv.load_dotenv()

connection_string=os.getenv('connection_string')
container_name='pdf'
from azure.storage.blob import BlobServiceClient 

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)
# Settings
idrssds = [25647,208244,277240,285348,347639,608844,812436,839572,936136,2316714,2343167,2805535,2869162]
report_date = "03/31/2025"
download_dir = os.path.abspath("downloads")
os.makedirs(download_dir,exist_ok=True)

options = Options()
# comment below during debugging
# options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
prefs = {
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True,
    "download.default_directory": download_dir,  #  download here
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True  #  skip PDF viewer
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 30)

for idrss in idrssds:
    driver.get("https://cdr.ffiec.gov/public/ManageFacsimiles.aspx")
    print("URL:", driver.current_url)
    print("TITLE:", driver.title)

    try:
        Select(wait.until(EC.presence_of_element_located((By.ID, 'reportTypeDropDownList')))).select_by_visible_text('Call Report')
        # wait.until(EC.presence_of_element_located((By.ID, 'reportTypeDropDownList')))
    except:
        print("Report dropdown not found. Saving page...")
        with open("debug_page.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        driver.quit()
        raise

    date_dropdown = Select(wait.until(EC.presence_of_element_located((By.ID, 'selectDateFromCall'))))
    date_dropdown.select_by_visible_text(report_date)

    id_input = wait.until(EC.presence_of_element_located((By.ID, 'txtUniqueIdentifierCall')))
    id_input.clear()
    id_input.send_keys(str(idrss))

    gen_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnGenerateByIdentifierCall')))
    gen_button.click()

    # Wait for PDF tab to open
    time.sleep(5)
    windows = driver.window_handles
    if len(windows) > 1:
        driver.switch_to.window(windows[-1])
        wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Download PDF']"))).click()
        # Wait for download to complete
        time.sleep(8)  # increase if PDF is large
        driver.close()
        driver.switch_to.window(windows[0])

        # Find the downloaded PDF file
        expected_filename = f"CallReport_{idrss}_{report_date.replace('/', '')}.pdf"
        # downloaded_files = os.listdir(download_dir)
        # downloaded_pdf = next((f for f in downloaded_files if f.lower().endswith('.pdf')), None)
        pdf_files = [
            os.path.join(download_dir, f)
            for f in os.listdir(download_dir)
            if f.lower().endswith(".pdf")
        ]

        # Sort by modification time (latest last)
        pdf_files.sort(key=os.path.getmtime, reverse=True)
        downloaded_pdf = pdf_files[0] if pdf_files else None
        print(downloaded_pdf)
        if downloaded_pdf:
            local_path = os.path.join(download_dir, downloaded_pdf)
            blob_client = container_client.get_blob_client(expected_filename)

            with open(local_path, 'rb') as data:
                blob_client.upload_blob(data, overwrite=True)

            print(f" Uploaded to Blob: {expected_filename}")
       
        else:
            print(f" PDF download not detected for IDRSSD: {idrss}")
    else:
        print(f" No new window/tab opened for IDRSSD: {idrss}")

driver.quit()

URL: https://cdr.ffiec.gov/public/ManageFacsimiles.aspx
TITLE: View or download data for individual institutions - FFIEC Central Data Repository's Public Data Distribution
d:\pythonProjects\Langgraph-Agent-course\Peer_Analysis\downloads\Call_Cert22536_033125 (1).PDF
 Uploaded to Blob: CallReport_25647_03312025.pdf
URL: https://cdr.ffiec.gov/public/ManageFacsimiles.aspx
TITLE: View or download data for individual institutions - FFIEC Central Data Repository's Public Data Distribution
d:\pythonProjects\Langgraph-Agent-course\Peer_Analysis\downloads\Call_Cert3832_033125 (1).PDF
 Uploaded to Blob: CallReport_208244_03312025.pdf
URL: https://cdr.ffiec.gov/public/ManageFacsimiles.aspx
TITLE: View or download data for individual institutions - FFIEC Central Data Repository's Public Data Distribution
d:\pythonProjects\Langgraph-Agent-course\Peer_Analysis\downloads\Call_Cert22282_033125 (1).PDF
 Uploaded to Blob: CallReport_277240_03312025.pdf
URL: https://cdr.ffiec.gov/public/ManageFacsimiles.

# Automation task 2

1. read the pdfs one by one 
2. use PyMuPDF/pdfplumber/pdfminer to extract raw text 
3. use azure open AI API and setup prompt 
4. create a pipeline code


In [10]:
# version 3 
import os
import json
import time
import fitz  # PyMuPDF
from azure.storage.blob import BlobServiceClient
from openai import AzureOpenAI, RateLimitError, APIError
import dotenv 
import math 

dotenv.load_dotenv()
records=[]
# OpenAI client setup
openai_client = AzureOpenAI(
    azure_deployment='gpt-35-turbo',
    api_key=os.getenv('AZURE_OPENAI_GPT_35_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_GPT_35_API_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_GPT_35_API_VERSION')
)

# Blob setup
blob_conn_str = "DefaultEndpointsProtocol=https;AccountName=peeranalysis;AccountKey=tlHuAeMbU1AOf3iMcyHkLIYWVoyfAAVzSbxp2TuvYFIAtsXnX9wJC+Q4u4dpBc6qm3ZmzCYmSaFH+AStluqCeg==;EndpointSuffix=core.windows.net"
container_name = "pdf"
deployment = "gpt-35-turbo"

extraction_prompt = 		"""
		You are a financial document parser. Extract the following 20 specific fields from this U.S. bank call report text. Only extract values explicitly shown in the text. Do not guess, infer, or calculate unless explicitly asked.

If a value is missing, not reported, unreadable, or appears as a placeholder (e.g., "Click here", "-", "...", "none", "n/a", etc.), return "NR".

Fields to extract:

- "IDRSSD"
- "Financial Institution Name"
- "Financial Institution Address"
- "Financial Institution City"
- "Financial Institution State"
- "Financial Institution Zip Code"
- "Submission Date" (format: YYYY-MM-DD HH:MM:SS; if no time, use 00:00:00)
- "Financial Institution Filing Type" (e.g., 041, 051)
- "Total Assets"
- "Total Liabilities"
- "Total Equity"
- "Total Revenue"
- "Total Expenses"
- "Total Deposits"
- "Total Loans"
- "Net Income"
- "Net Charge-Offs"
- "Loan Loss Reserves (LLR)"
- "Total Non-Performing Loans (NPLs)"
- "LLR/NPL" (only if both LLR and NPL are numeric and NPL ≠ 0, otherwise use "inf" or "NR")

Example:
{
  "IDRSSD": "123456",
  "Financial Institution Name": "SAMPLE BANK",
  "Financial Institution Address": "456 MAIN STREET",
  ...
  "LLR/NPL": "0.85"
}

Now extract the values from the following text. Return only the JSON:
"""


def call_openai_with_retry(messages, retries=5):
    delay = 10
    for attempt in range(retries):
        try:
            return openai_client.chat.completions.create(
                model=deployment,
                messages=messages,
                temperature=0.0
            )
        except (RateLimitError, APIError) as e:
            print(f"Rate limit or API error: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2
    raise Exception("Failed after multiple retries.")

def split_text_by_pages(doc, pages_per_chunk=5):
    chunks = []
    for i in range(0, len(doc), pages_per_chunk):
        chunk = "\n".join(doc[j].get_text() for j in range(i, min(i + pages_per_chunk, len(doc))))
        chunks.append(chunk)
    return chunks

def is_invalid(value):
    return (
        value in ("", "NR", None) or
        (isinstance(value, float) and math.isnan(value))
    )

def safe_json_merge(base, new):
    try:
        for k, v in new.items():
            if k not in base or is_invalid(base[k]):
                if not is_invalid(v):  # only assign if new value is valid
                    base[k] = v
    except Exception as e:
        print(f" Error merging JSON: {e}")
    return base

# def safe_json_merge(base, new):
#     try:
#         base.update({k: v for k, v in new.items() if v})  # only overwrite if new value is non-empty
#     except Exception as e:
#         print(f" Error merging JSON: {e}")
#     return base

# Connect to blob container
blob_service_client = BlobServiceClient.from_connection_string(blob_conn_str)
container_client = blob_service_client.get_container_client(container_name)

# Loop through PDFs
for blob in container_client.list_blobs():
    print(f"\n Processing {blob.name}...")
    blob_client = container_client.get_blob_client(blob)
    pdf_content = blob_client.download_blob().readall()

    # Save and open PDF
    with open("temp.pdf", "wb") as f:
        f.write(pdf_content)
    doc = fitz.open("temp.pdf")

    text_chunks = split_text_by_pages(doc, pages_per_chunk=10)
    merged_data = {}

    for i, chunk in enumerate(text_chunks):
        messages = [
            {"role": "system", "content": extraction_prompt},
            {"role": "user", "content": chunk}
        ]
        response = call_openai_with_retry(messages)
        try:
            content = response.choices[0].message.content
            extracted = json.loads(content)
            if extracted:
                successful_extraction=True
                merged_data = safe_json_merge(merged_data, extracted)
                print(f" Raw response from chunk {i+1}:\n{merged_data}\n")
            
        except Exception as e:
            print(f"Failed to parse JSON in chunk {i+1}: {e}")
            continue
    # extracted = json.loads(merged_data)
    records.append(merged_data)
    print(f"\n Final extracted data from {blob.name}:\n{json.dumps(merged_data, indent=2)}\n")



 Processing CallReport_208244_03312025.pdf...
 Raw response from chunk 1:
{'IDRSSD': '208244', 'Financial Institution Name': 'OLD NATIONAL BANK', 'Financial Institution City': 'EVANSVILLE', 'Financial Institution State': 'IN', 'Financial Institution Zip Code': '47708', 'Submission Date': '2025-03-31 00:00:00', 'Financial Institution Filing Type': '031', 'Net Income': '156314', 'Net Charge-Offs': '31403'}

 Raw response from chunk 2:
{'IDRSSD': '208244', 'Financial Institution Name': 'OLD NATIONAL BANK', 'Financial Institution City': 'EVANSVILLE', 'Financial Institution State': 'IN', 'Financial Institution Zip Code': '47708', 'Submission Date': '2025-03-31 00:00:00', 'Financial Institution Filing Type': '031', 'Net Income': '156314', 'Net Charge-Offs': '31403', 'Total Assets': '53,574,412', 'Total Liabilities': '47,231,510', 'Total Equity': '6,342,902', 'Total Deposits': '41,333,273', 'Total Loans': '36,413,944', 'Loan Loss Reserves (LLR)': '401,932', 'LLR/NPL': 'inf'}

 Raw response f

In [11]:
import pandas as pd 
df = pd.DataFrame(records)
print("\nFinal Extracted DataFrame:")
print(df.head())
df.to_csv("extracted_call_report_data.csv", index=False)



Final Extracted DataFrame:
    IDRSSD       Financial Institution Name Financial Institution City  \
0   208244                OLD NATIONAL BANK                 EVANSVILLE   
1  2316714                        TIME BANK                 PARK RIDGE   
2  2343167        PAN AMERICAN BANK & TRUST               MELROSE PARK   
3    25647  FIRST SECURE BANK AND TRUST CO.                PALOS HILLS   
4   277240               FIRST NATIONS BANK                    CHICAGO   

  Financial Institution State Financial Institution Zip Code  \
0                          IN                          47708   
1                          IL                          60068   
2                          IL                          60160   
3                          IL                          60465   
4                          IL                          60631   

       Submission Date Financial Institution Filing Type Net Income  \
0  2025-03-31 00:00:00                               031     156314   


In [ ]:
def fetch_address(row):
    name = row['Financial Institution Name']
    city = row['Financial Institution City']
    state = row['Financial Institution State']
    zip_code = row['Financial Institution Zip Code']
    
    query = f"{name}, {city}, {state} {zip_code}"
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": query,
        "format": "json",
        "limit": 1
    }

    try:
        response = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            data = response.json()
            if data:
                return data[0].get("display_name", "NR")
    except Exception as e:
        print(f"Error fetching address for {query}: {e}")
    
    return "NR"

In [ ]:
# def compute_llr_npl(row):
#     try:
#         llr = float(str(row["Loan Loss Reserves (LLR)"]).replace(",", ""))
#         npl = float(str(row["Total Non-Performing Loans (NPLs)"]).replace(",", ""))
#         if npl == 0:
#             return "inf"
#         return round(llr / npl, 2)
#     except:
#         return "NR"

In [ ]:
# df["Financial Institution Address"] = df.apply(
#     lambda row: fetch_address(row) if row["Financial Institution Address"] in ["NR", "", None] else row["Financial Institution Address"],
#     axis=1
# )

# # Compute LLR/NPL field
# df["LLR/NPL"] = df.apply(compute_llr_npl, axis=1)

In [ ]:
# Optional: Save to CSV
df.to_csv("extracted_call_report_data.csv", index=False)